In [ ]:
from keras.models import Sequential
from keras.layers.convolutional import Conv3D
from keras.layers.convolutional_recurrent import ConvLSTM2D
from keras.layers.normalization import BatchNormalization
from keras.layers.core import Activation, Dropout, Dense
from keras.layers import Flatten, LSTM
import numpy as np
import pylab as plt
import tensorflow as tf
from sklearn.preprocessing import normalize
import os,glob


In [ ]:
input_file = '/Users/mylesfoley/git/4thYearProject/0000.txt'

#image_list = glob.glob('{:s}/**/*.jpg'.format(src_dir), recursive=True)
#input_data = []
input_data = {}
with open(input_file, 'r') as data_stream:
    for line in data_stream:
        current_line = line.split(' ')
        if not (('DontCare' in current_line[2]) or ('Pedestrian' in current_line[2]) or 
                ('Cyclist' in current_line[2])):
            if (current_line[2]+ current_line[1]) in input_data.keys():
                #key by object and unique identifier e.g. Car1
                input_data[current_line[2] + current_line[1]].append({'frame': current_line[0], 
                     'bbox': current_line[6:10]})
            else:
                input_data[current_line[2] + current_line[1]] = [{'frame': current_line[0], 
                     'bbox': current_line[6:10]}]

In [ ]:
X = []
y = []
list_lengths = []
for unique_item in input_data:
    item_list = []
    for frame_entry in input_data[unique_item]:
        item_list.append(frame_entry['bbox'])
    list_lengths.append(len(item_list))
    if len(item_list) > 27:
        X.append(np.array(item_list[:28]))
        y.append(np.array(item_list[:28]))
X = np.array(X)
y = np.array(y)

In [ ]:
Xnorm = []
ynorm = []
for sample in range(0,X.shape[0]):
    Xnorm.append(normalize(X[sample,:], axis=0)[:27])
    
    ynorm.append(normalize(y[sample, :], axis=0)[1:28])

In [ ]:
ytr = np.array(ynorm[:8])
Xtr = np.array(Xnorm[:8])
yte = np.array(ynorm[8:9])
Xte = np.array(Xnorm[8:9])

In [ ]:
modelItem = Sequential()
modelItem.add(LSTM(32, return_sequences = True,input_shape=(27, 4), activation='relu'))
modelItem.add(Dropout(0.2))

modelItem.add(LSTM(units = 50, return_sequences = True, activation='relu'))
modelItem.add(Dropout(0.2))

modelItem.add(LSTM(units = 50, return_sequences = True, activation='relu'))
modelItem.add(Dropout(0.2))


modelItem.add(Dense(4))
modelItem.compile(optimizer='adam', loss='mse')

In [ ]:
history = modelItem.fit(Xtr, ytr, epochs=500, validation_split=0.2, verbose=1)

In [ ]:
modelItem.predict(Xte, verbose=0)

In [ ]:
yte